<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [4]:
## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [12]:
import folium
from folium.features import DivIcon

# Initialize the map centered at NASA coordinate
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Extract lat/long and launch site names
lats_longs = list(zip(launch_sites_df['Lat'], launch_sites_df['Long']))
locations = list(launch_sites_df['Launch Site'])

# Add a circle and a label for each launch site
for i in range(len(locations)):
    coord = lats_longs[i]
    
    # Add circle marker
    circle = folium.Circle(
        location=coord,
        radius=1000,
        color='#d35400',
        fill=True
    ).add_child(folium.Popup(f'{locations[i]}'))
    
    # Add text label using DivIcon at the same coordinate
    marker = folium.map.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{locations[i]}</b></div>'
        )
    )
    
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map (in Jupyter)
site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [14]:
import folium
from folium import Marker
from folium.plugins import MarkerCluster

# Optional: Grouping markers for readability
marker_cluster = MarkerCluster()

# Create base map centered around Kennedy Space Center
nasa_coordinate = [28.573255, -80.646895]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Loop through each row in the dataframe
for index, row in spacex_df.iterrows():
    coord = [row['Lat'], row['Long']]
    site = row['Launch Site']
    outcome = 'Success' if row['class'] == 1 else 'Failure'
    color = 'green' if row['class'] == 1 else 'red'
    
    # Create marker with popup and colored icon
    marker = folium.Marker(
        location=coord,
        popup=f"{site} - {outcome}",
        icon=folium.Icon(color=color, icon='rocket', prefix='fa')
    )
    
    # Add to cluster
    marker_cluster.add_child(marker)

# Add cluster to map
site_map.add_child(marker_cluster)

# Show the map
site_map



Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [15]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [16]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [17]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
import numpy as np

spacex_df['marker_color'] = np.where(spacex_df['class'] == 1, 'green', 'red')


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [18]:
import folium
from folium.plugins import MarkerCluster

# Create the map and marker cluster
nasa_coordinate = [28.573255, -80.646895]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
marker_cluster = MarkerCluster()

# Loop through the DataFrame and add markers
for index, record in spacex_df.iterrows():
    coord = [record['Lat'], record['Long']]
    site = record['Launch Site']
    outcome = 'Success' if record['class'] == 1 else 'Failure'
    
    # Create and add the Marker
    marker = folium.Marker(
        location=coord,
        popup=f"{site} - {outcome}",
        icon=folium.Icon(color='white', icon='rocket', icon_color=record['marker_color'])
    )
    marker_cluster.add_child(marker)

# Add the marker cluster to the map
site_map.add_child(marker_cluster)

# Display the map
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [21]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import math

# Read the SpaceX launch data
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df = pd.read_csv(spacex_csv_file)

# STEP 1: Create marker_color column
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# STEP 2: Create folium map and MarkerCluster
nasa_coordinate = [28.573255, -80.646895]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
marker_cluster = MarkerCluster()

# STEP 3: Add launch markers with custom icon color
for index, record in spacex_df.iterrows():
    coord = [record['Lat'], record['Long']]
    site = record['Launch Site']
    outcome = 'Success' if record['class'] == 1 else 'Failure'
    
    marker = folium.Marker(
        location=coord,
        popup=f"{site} - {outcome}",
        icon=folium.Icon(color='white', icon='rocket', icon_color=record['marker_color'])
    )
    marker_cluster.add_child(marker)

site_map.add_child(marker_cluster)

# STEP 4: Unique launch sites
launch_sites = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates().reset_index(drop=True)

# STEP 5: Define proximity landmarks
proximity_points = {
    'Cocoa Beach (City)': (28.3922, -80.6077),
    'Closest Coast': (28.561857, -80.567992),
    'Highway Intersection': (28.562302, -80.577356)
}

# STEP 6: Haversine function
def haversine(coord1, coord2):
    R = 6371
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# STEP 7: Calculate distances
for label, prox_coord in proximity_points.items():
    launch_sites[f'Distance to {label} (km)'] = launch_sites.apply(
        lambda row: haversine((row['Lat'], row['Long']), prox_coord), axis=1
    )

# Display the final DataFrame with distances
launch_sites


,Launch Site,Lat,Long,Distance to Cocoa Beach (City) (km),Distance to Closest Coast (km),Distance to Highway Intersection (km)
0,CCAFS LC-40,28.562302,-80.577356,19.145585,0.915896,0.000047
1,VAFB SLC-4E,34.632834,-120.610745,3829.725900,3826.714220,3825.840322
2,KSC LC-39A,28.573255,-80.646895,20.493571,7.809016,6.899350
3,CCAFS SLC-40,28.563197,-80.576820,19.252080,0.874948,0.112463


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [22]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [23]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [24]:
import pandas as pd
import math

# Load the SpaceX dataset
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df = pd.read_csv(spacex_csv_file)

# Define the Haversine distance function
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c

# Define the closest coastline coordinates
coastline_lat = 28.56367
coastline_lon = -80.57163

# Get unique launch sites
launch_sites = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates().reset_index(drop=True)

# Calculate the distance from each launch site to the coastline
launch_sites['Distance to Coastline (km)'] = launch_sites.apply(
    lambda row: calculate_distance(row['Lat'], row['Long'], coastline_lat, coastline_lon),
    axis=1
)

# Display the result
launch_sites


,Launch Site,Lat,Long,Distance to Coastline (km)
0,CCAFS LC-40,28.562302,-80.577356,0.579576
1,VAFB SLC-4E,34.632834,-120.610745,3826.310821
2,KSC LC-39A,28.573255,-80.646895,7.427012
3,CCAFS SLC-40,28.563197,-80.576820,0.509584


In [25]:
import pandas as pd
import math
import folium
from folium.features import DivIcon

# Load the SpaceX dataset
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df = pd.read_csv(spacex_csv_file)

# Define Haversine distance function
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Closest coastline coordinates
coastline_lat = 28.56367
coastline_lon = -80.57163
coastline_coord = (coastline_lat, coastline_lon)

# Get unique launch sites
launch_sites = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates().reset_index(drop=True)

# For this example, pick the first launch site
launch_site_name = launch_sites.loc[0, 'Launch Site']
launch_lat = launch_sites.loc[0, 'Lat']
launch_lon = launch_sites.loc[0, 'Long']
launch_coord = (launch_lat, launch_lon)

# Calculate distance to coastline
distance = calculate_distance(launch_lat, launch_lon, coastline_lat, coastline_lon)

# Create base map centered at launch site
site_map = folium.Map(location=launch_coord, zoom_start=12)

# Add marker for launch site
folium.Marker(
    location=launch_coord,
    popup=f"Launch Site: {launch_site_name}",
    icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
).add_to(site_map)

# Add marker for coastline point
folium.Marker(
    location=coastline_coord,
    popup="Closest Coastline",
    icon=folium.Icon(color='green')
).add_to(site_map)

# Add distance label as DivIcon (text marker)
folium.Marker(
    location=[(launch_lat + coastline_lat)/2, (launch_lon + coastline_lon)/2],  # midpoint
    icon=DivIcon(
        icon_size=(150, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
    )
).add_to(site_map)

# Display the map
site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [26]:
import pandas as pd
import math
import folium
from folium.features import DivIcon

# Load the SpaceX dataset
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df = pd.read_csv(spacex_csv_file)

# Haversine distance function
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # km
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Closest coastline coordinates
coastline_lat = 28.56367
coastline_lon = -80.57163
coastline_coord = (coastline_lat, coastline_lon)

# Get unique launch sites
launch_sites = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates().reset_index(drop=True)

# Pick the first launch site for this example
launch_site_name = launch_sites.loc[0, 'Launch Site']
launch_lat = launch_sites.loc[0, 'Lat']
launch_lon = launch_sites.loc[0, 'Long']
launch_coord = (launch_lat, launch_lon)

# Calculate distance
distance = calculate_distance(launch_lat, launch_lon, coastline_lat, coastline_lon)

# Initialize map centered at launch site
site_map = folium.Map(location=launch_coord, zoom_start=12)

# Add launch site marker
folium.Marker(
    location=launch_coord,
    popup=f"Launch Site: {launch_site_name}",
    icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
).add_to(site_map)

# Add coastline marker
folium.Marker(
    location=coastline_coord,
    popup="Closest Coastline",
    icon=folium.Icon(color='green')
).add_to(site_map)

# Add distance label at midpoint
folium.Marker(
    location=[(launch_lat + coastline_lat)/2, (launch_lon + coastline_lon)/2],
    icon=DivIcon(
        icon_size=(150, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
    )
).add_to(site_map)

# Add a polyline (line) between launch site and coastline
line = folium.PolyLine(
    locations=[launch_coord, coastline_coord],
    weight=2,
    color='orange',
    dash_array='5, 5'
)
site_map.add_child(line)

# Show the map
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [27]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
import pandas as pd
import math
import folium
from folium.features import DivIcon

# Load the SpaceX dataset
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df = pd.read_csv(spacex_csv_file)

# Haversine distance function
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # km
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Define proximity coordinates (you can replace with actual ones)
proximities = {
    "Closest City": (28.3922, -80.6077),            # Cocoa Beach
    "Closest Railway": (28.5721, -80.5853),          # Example railway
    "Closest Highway": (28.5623, -80.5774)           # Example highway
}

# Get the first launch site
launch_sites = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates().reset_index(drop=True)
launch_site_name = launch_sites.loc[0, 'Launch Site']
launch_lat = launch_sites.loc[0, 'Lat']
launch_lon = launch_sites.loc[0, 'Long']
launch_coord = (launch_lat, launch_lon)

# Initialize map
site_map = folium.Map(location=launch_coord, zoom_start=12)

# Add launch site marker
folium.Marker(
    location=launch_coord,
    popup=f"Launch Site: {launch_site_name}",
    icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
).add_to(site_map)

# Loop through each proximity feature
for label, prox_coord in proximities.items():
    # Calculate distance
    distance = calculate_distance(launch_lat, launch_lon, prox_coord[0], prox_coord[1])

    # Add proximity marker
    folium.Marker(
        location=prox_coord,
        popup=label,
        icon=folium.Icon(color='green')
    ).add_to(site_map)

    # Add distance label at midpoint
    mid_lat = (launch_lat + prox_coord[0]) / 2
    mid_lon = (launch_lon + prox_coord[1]) / 2

    folium.Marker(
        location=[mid_lat, mid_lon],
        icon=DivIcon(
            icon_size=(150, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % "{:.2f} KM".format(distance),
        )
    ).add_to(site_map)

    # Draw line from launch site to proximity point
    line = folium.PolyLine(
        locations=[launch_coord, prox_coord],
        weight=2,
        color='purple',
        dash_array='5, 5'
    )
    site_map.add_child(line)

# Show map
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
